# Training FastText
1. Load data to a Spark dataframe
2. Store data to a training file
3. Train a fastText model for language prediction
4. Evaluate the model (#3.-Train-a-fastText-model-for-language-prediction)[link]

In [1]:
spark

NameError: name 'spark' is not defined

In [3]:
#from pyspark.sql import SparkSession
#from pyspark.sql.functions import col, udf
from pyspark.sql.types import *
from pyspark.sql.types import StructType

import fasttext

## 1. Load data to Spark dataframe

In [4]:
# Define schema
schema = StructType([
    StructField("sentence_id", IntegerType(), True),
    StructField("language_code", StringType(), True),
    StructField("text", StringType(), True)])

In [5]:
# Read CSV into spark dataframe
spark_df = spark.read.csv('data/sentences.csv', schema=schema, sep='\t')

In [2]:
# Verify that the our schema looks valid
spark_df.printSchema()

NameError: name 'spark_df' is not defined

In [7]:
# Show 20 samples
spark_df.show()

+-----------+-------------+--------------------+
|sentence_id|language_code|                text|
+-----------+-------------+--------------------+
|          1|          cmn|              我們試試看！|
|          2|          cmn|             我该去睡觉了。|
|          3|          cmn|             你在干什麼啊？|
|          4|          cmn|              這是什麼啊？|
|          5|          cmn|今天是６月１８号，也是Muirie...|
|          6|          cmn|       生日快乐，Muiriel！|
|          7|          cmn|      Muiriel现在20岁了。|
|          8|          cmn|       密码是"Muiriel"。|
|          9|          cmn|            我很快就會回來。|
|         10|          cmn|               我不知道。|
|         11|          cmn|        我不知道應該說什麼才好。|
|         12|          cmn|           這個永遠完不了了。|
|         13|          cmn|     我只是不知道應該說什麼而已……|
|         14|          cmn|         那是一隻有惡意的兔子。|
|         15|          cmn|             我以前在山里。|
|         16|          cmn|            那是一张近照吗？|
|         17|          cmn|         我不知道我有沒有時間。|
|         18|       

# 2. Store data to a training file

In [9]:
TRAIN_FILE = 'data/fasttext_train.txt'
TEST_FILE = 'data/fasttext_test.txt'
MODEL_FILE = 'data/fasttext_language'

## Split data into train and test

In [10]:
train_test_df = spark_df.randomSplit([0.8,0.2], 42)

In [11]:
# Does not compute. Task size too big (98382 KB > 100 KB.)
print(train_test_df[0].count())
print(train_test_df[1].count())

4780503
1196155


## Write training and testing set files

In [12]:
with open(TRAIN_FILE, 'w') as fp:
    for i, row in enumerate(train_test_df[0].toLocalIterator()):
        fp.write('__label__%s %s\n' % (row['language_code'],
                                             row['text']))
        if i % 1000 == 0:
            print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [ ]:
with open(TEST_FILE, 'w') as fp:
    for i, row in enumerate(train_test_df[1].toLocalIterator()):
        fp.write('__label__%s %s\n' % (row['language_code'],
                                             row['text']))
        if i % 1000 == 0:
            print(i)

## Hacky way to split rows in train and test

In [ ]:
fp_train = open(TRAIN_FILE, 'w')
fp_test = open(TEST_FILE, 'w')

for i, row in enumerate(spark_df.toLocalIterator()):
    if ((i % 10) == 9): #or ((i % 10) == 8):
        fp_test.write('__label__%s %s\n' % (row['language_code'], row['text']))
    else:
        fp_train.write('__label__%s %s\n' % (row['language_code'], row['text']))
    if i % 1000 == 0:
        print(i)

## 3. Train a fastText model for language prediction

In [ ]:
model = fasttext.supervised(TRAIN_FILE, MODEL_FILE)

In [ ]:
model.predict_proba(['Hello world'])

## 4. Evaluate the model